Assumptions and initial data are expressed below

In [ ]:
import math

# Constants
SpecificHeatCapacityOfWater = 4186  # J/kg·K
LatentHeatOfFusion = 334000         # J/kg
SpecificHeatCapacityOfIce = 2100    # J/kg·K
DensityOfWater = 1000               # kg/m³

def CalculateTimeToTemperature(WaterInitTemp, WaterFinalTemp, AmbientTemp, InnerVolume, WallThickness, HeatConductivity):
    # Derived properties
    r_inner = ((3 * InnerVolume) / (4 * math.pi)) ** (1/3)
    r_outer = r_inner + WallThickness
    WaterMass = InnerVolume * DensityOfWater

    # Energy required in each phase
    EnergyToCoolWater = WaterMass * SpecificHeatCapacityOfWater * max(0, WaterInitTemp - 0)
    EnergyToFreeze = WaterMass * LatentHeatOfFusion
    EnergyToCoolIce = WaterMass * SpecificHeatCapacityOfIce * max(0, 0 - WaterFinalTemp)
    TotalEnergyToRemove = EnergyToCoolWater + EnergyToFreeze + EnergyToCoolIce

    # Time stepping
    time = 0  # seconds
    time_step = 60  # seconds
    energy_removed = 0

    while energy_removed < TotalEnergyToRemove:
        # Estimate current internal temperature
        if energy_removed < EnergyToCoolWater:
            current_temp = WaterInitTemp - (energy_removed / EnergyToCoolWater) * (WaterInitTemp - 0)
        elif energy_removed < (EnergyToCoolWater + EnergyToFreeze):
            current_temp = 0  # Freezing stage
        else:
            frac = (energy_removed - EnergyToCoolWater - EnergyToFreeze) / EnergyToCoolIce
            current_temp = 0 + frac * (WaterFinalTemp - 0)

        # Accurate radial conduction for a spherical shell
        Q = 4 * math.pi * HeatConductivity * (r_inner * r_outer) / (r_outer - r_inner) * (current_temp - AmbientTemp)

        energy_this_step = Q * time_step
        energy_removed += energy_this_step
        time += time_step

    return time / 3600  # Convert to hours

# Example usage
WaterInitTemp = 20  # °C
WaterFinalTemp = -80  # °C
AmbientTemp = -80  # °C
InnerVolume = 0.0041  # m³ (4.1 liters)
WallThickness = 0.07  # m
HeatConductivity = 0.033  # W/m·K (styrofoam)

hours = CalculateTimeToTemperature(WaterInitTemp, WaterFinalTemp, AmbientTemp, InnerVolume, WallThickness, HeatConductivity)
print(f"Time (hours): {hours:.2f}")


KeyboardInterrupt: 